In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
stats = pd.read_csv(os.path.join("csv/MLB DB V1 CSV.csv"))
stats.rename(columns={"O/U Odds": "O/U_Odds"}, inplace= True)

stats.FIELD[stats.FIELD == 'HOME'] = 1
stats.FIELD[stats.FIELD == 'AWAY'] = 0

stats = stats.drop(columns = ["Team", "Season", "Date", "Vs", "Score", "Away Starter", "Home Starter", "BAL Line", "O/U", "O/U Result", "Unnamed: 9", "Margin", "Runs Scr", "Runs Alw"])

stats= stats.dropna()

stats.head()

,FIELD,Game Result,Side Line,O/U Line,O/U_Odds,Results on the ML,Results on the Over,Results on the Under
0,1,L,-155.0,8.5,-106,-$1.00,-$1.00,$1.94
1,1,W,-155.0,8.5,-116,$1.65,-$1.00,$1.86
2,1,W,-145.0,8.0,-106,$1.69,-$1.00,$1.94
3,1,W,-120.0,9.0,107,$1.83,$2.07,-$1.00
4,0,W,176.0,9.0,-107,$2.76,-$1.00,$1.93


In [207]:
stats.drop(stats[stats["O/U_Odds"] == "9 -"].index, inplace=True)
stats.drop(stats[stats["O/U_Odds"] == "7 -"].index, inplace=True)
stats.drop(stats[stats["O/U_Odds"] == "8 -"].index, inplace=True)
stats.drop(stats[stats["O/U_Odds"] == "10 -"].index, inplace=True)
stats.drop(stats[stats["O/U_Odds"] == "11 -"].index, inplace=True)
stats.drop(stats[stats["O/U_Odds"] == "6 -"].index, inplace=True)

In [208]:
stats.FIELD[stats.FIELD == 'HOME'] = 1
stats.FIELD[stats.FIELD == 'AWAY'] = 0

In [209]:
# Assign X (data) and y (target)
X = stats.drop("Game Result", axis=1)
y = stats["Game Result"]
print(X.shape, y.shape)

(4690, 4) (4690,)


In [210]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [211]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression()

In [212]:
classifier.fit(X_train, y_train)

LogisticRegression()

In [213]:
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.5769121410292863
Testing Data Score: 0.5797101449275363


In [214]:
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   ['L' 'L' 'W' 'W' 'L' 'W' 'W' 'W' 'W' 'L']
First 10 Actual labels: ['L', 'W', 'W', 'W', 'W', 'L', 'W', 'W', 'W', 'W']


In [215]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,L,L
1,L,W
2,W,W
3,W,W
4,L,W
...,...,...
1168,W,W
1169,L,L
1170,W,W
1171,L,L


In [216]:
stats.to_csv(r'C:\Users\Jackson\Documents\Bootcamp\Final Project\indiana_data_farmers\Stats.csv', index = False) 